In [ ]:
#! pip install 'h5py==2.10.0' --force-reinstall

In [1]:
%tensorflow_version 1.x


from google.colab import drive
drive.mount("/content/drive")

import sys

PATH = '/content/drive/MyDrive/Colab Notebooks/CS598_DLH_Paper211'
sys.path.append(PATH)


TensorFlow 1.x selected.
Mounted at /content/drive


In [3]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
# import glove
# from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, merge, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, merge

from keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
from keras.backend.tensorflow_backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

SEED='normal'

Using TensorFlow backend.


In [ ]:
# Reset Keras Session
def reset_keras(model):
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    gc.collect() # if it's done something you should see a number being outputted

def make_prediction_timeseries(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_timeseries(predictions, probs, ground_truth, model_name, 
                problem_type, iteration, hidden_unit_size, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1

        
    file_name = str(hidden_unit_size)+"-"+model_name+"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+".p"
    
    result_path = PATH + f"/results/TimeSeriesBaseline/{SEED}/"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)

In [ ]:
def timeseries_model(layer_name, number_of_unit):
    K.clear_session()
    
    sequence_input = Input(shape=(24,104),  name = "timeseries_input")
    
    if layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)
    else:
        x = GRU(number_of_unit)(sequence_input)
    
    logits_regularizer = tf.contrib.layers.l2_regularizer(scale=0.01)
    sigmoid_pred = Dense(1, activation='sigmoid',use_bias=False, kernel_regularizer=logits_regularizer)(x)
    model = Model(inputs=sequence_input, outputs=sigmoid_pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [ ]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle(PATH + "/data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle(PATH + "/data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle(PATH + "/data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle(PATH + "/data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle(PATH + "/data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle(PATH + "/data/"+type_of_ner+"_y_test.pkl")

In [ ]:
epoch_num = 50
model_patience = 3
monitor_criteria = 'val_loss'
batch_size = 128

unit_sizes = [128, 256]
unit_sizes = [256]
iter_num = 11

#Changed target to include additonal experiments
target_problems = ['mort_hosp', 'mort_icu', 'los_3', 'los_7']
target_problems = ['los_2', 'los_4','los_5']

# layers = ["LSTM", "GRU"]
layers = ["GRU"]
for each_layer in layers:
    print("Layer: ", each_layer)
    for each_unit_size in unit_sizes:
        print("Hidden unit: ", each_unit_size)
        for iteration in range(1, iter_num):
            print("Iteration number: ", iteration)
            print("=============================")

            for each_problem in target_problems:
                print ("Problem type: ", each_problem)
                print ("__________________")


                early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
                best_model_name = str(each_layer)+"-"+str(each_unit_size)+"-"+str(each_problem)+"-"+"best_model.hdf5"
                checkpoint = ModelCheckpoint(best_model_name, monitor='val_loss', verbose=1,
                    save_best_only=True, mode='min', period=1)


                callbacks = [early_stopping_monitor, checkpoint]
                print(each_layer, each_unit_size)
                model = timeseries_model(each_layer, each_unit_size)
                model.fit(x_train_lstm, y_train[each_problem], epochs=epoch_num, verbose=1, 
                          validation_data=(x_dev_lstm, y_dev[each_problem]), callbacks=callbacks, batch_size= batch_size)

                model.load_weights(best_model_name)

                probs, predictions = make_prediction_timeseries(model, x_test_lstm)
                save_scores_timeseries(predictions, probs, y_test[each_problem].values,str(each_layer),
                                       each_problem, iteration, each_unit_size,type_of_ner)
                reset_keras(model)
                #del model
                clear_session()
                gc.collect()

In [ ]:
# ! pip install 'h5py==2.10.0' --force-reinstall


In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/CS598_DLH_Paper211/results/128-GRU-los_3-10-new.p')
df

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/CS598_DLH_Paper211/results/128-GRU-los_7-10-new.p')
df

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/CS598_DLH_Paper211/results/128-GRU-mort_hosp-10-new.p')
df

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/CS598_DLH_Paper211/results/128-GRU-mort_icu-10-new.p')
df

In [4]:
PATH = '/content/drive/MyDrive/Colab Notebooks/CS598_DLH_Paper211/results/TimeSeriesBaseline/normal'
subfolders = [ f.path for f in os.scandir(PATH) if f.is_dir() ]
subfolders

[]

In [5]:
import glob
from pathlib import Path

results_dict = {'target_problem':[],'F1':[],'acc':[],'auc':[],'auprc':[]}


files = glob.glob(f"{PATH}/*.p")
for file in files:
  file_name = Path(file).stem
  file_name_items = file_name.split('-')
  results_dict['target_problem'].append(file_name_items[2])
  scores_dict = pd.read_pickle(file)
  results_dict['acc'].append(scores_dict['acc'])
  results_dict['auc'].append(scores_dict['auc'])
  results_dict['auprc'].append(scores_dict['auprc'])
  results_dict['F1'].append(scores_dict['F1'])

In [6]:
df=pd.DataFrame.from_dict(results_dict,orient='index').transpose()
df.head()

,target_problem,F1,acc,auc,auprc
0,mort_hosp,0.456825,0.910304,0.87143,0.544004
1,mort_icu,0.44898,0.937902,0.880661,0.505418
2,los_3,0.551976,0.660994,0.694085,0.632072
3,los_7,0.02681,0.916513,0.704374,0.182408
4,mort_hosp,0.431085,0.910764,0.871573,0.551245


In [7]:
df_groupby = df.groupby(['target_problem']).agg({'auc':['mean'],'auprc':['mean'], 'F1':['mean']})

In [8]:
df_groupby_new = df.groupby(['target_problem']).agg({'auc':['mean'],'auprc':['mean'], 'F1':['mean']}).reset_index()

In [9]:
new_columns = []
for i in df_groupby_new.columns:
  j=''
  if i[1]:
    j='_'.join(list(i))
  else:
    j=i[0]
  new_columns.append(j)

df_groupby_new.columns = new_columns

In [10]:
df_groupby_new

,target_problem,auc_mean,auprc_mean,F1_mean
0,los_3,0.694074,0.635430,0.553869
1,los_7,0.720812,0.202277,0.055019
2,mort_hosp,0.872837,0.556042,0.447125
3,mort_icu,0.880599,0.504393,0.428067


In [11]:
PATH = '/content/drive/MyDrive/Colab Notebooks/CS598_DLH_Paper211/results/TimeSeriesBaseline/245'

In [12]:
import glob
from pathlib import Path

results_dict = {'target_problem':[],'F1':[],'acc':[],'auc':[],'auprc':[]}


files = glob.glob(f"{PATH}/*.p")
for file in files:
  file_name = Path(file).stem
  file_name_items = file_name.split('-')
  results_dict['target_problem'].append(file_name_items[2])
  scores_dict = pd.read_pickle(file)
  results_dict['acc'].append(scores_dict['acc'])
  results_dict['auc'].append(scores_dict['auc'])
  results_dict['auprc'].append(scores_dict['auprc'])
  results_dict['F1'].append(scores_dict['F1'])

In [13]:
df=pd.DataFrame.from_dict(results_dict,orient='index').transpose()
df_groupby = df.groupby(['target_problem']).agg({'auc':['mean'],'auprc':['mean'], 'F1':['mean']})
df_groupby_new = df.groupby(['target_problem']).agg({'auc':['mean'],'auprc':['mean'], 'F1':['mean']}).reset_index()

In [ ]:
new_columns = []
for i in df_groupby_new.columns:
  j=''
  if i[1]:
    j='_'.join(list(i))
  else:
    j=i[0]
  new_columns.append(j)

df_groupby_new.columns = new_columns

In [ ]:
df_groupby_new

,target_problem,auc_mean,auprc_mean,F1_mean
0,los_2,0.681898,0.837894,0.823623
1,los_4,0.703481,0.481999,0.364230
2,los_5,0.710476,0.367797,0.230184
